# Multirotor Altitude (Height) Control Mission

Welcome to this lab where we demonstrate cascade PI controllers in Python! Our mission is straightforward: maintain the altitude of a multirotor at 20 meters. To achieve this, we will implement two PI controllers. One will control the velocity of the multirotor, and the other will control its position (altitude).

By default, the multirotor in the Ubicoders virtual robot system operates without any system or sensor noise. For this demonstration, we'll use the default mode to focus on showcasing the cascade controller. If you'd like to turn on the noise, you can enable system noise from the simulation menu by pressing `Esc` -> Settings -> System Noise and/or Sensor Noise.

As this assumes the environment wihtout any noise, no sensor fusion is reuqired.

## 1. Getting Started

This environment is set up for your local machine. Please ensure you have the following:

1. Python 3.9+
2. `pip install ubicoders-vrobots ipython numpy matplotlib`

Optionally, if you're using VSCode:

3. Python extension for VSCode by Microsoft

## 2. Start the Bridge Servers

The virtual robot bridge server acts as a data communication bridge between the virtual robot webpage and this script. To control the robots, we need to run the bridge first.

Run the cell below and check the virtual robot webpage. A green light means it is ready to control the virtual robot! 

If not, you can:
1. Click the connect button
2. Or, refresh the page

<img src="https://raw.githubusercontent.com/ubicoders/yt_tutorials/main/images/tutorial1.png" alt="Description" style="max-width: 100%; width: 50%; height: auto;">

Let's dive in and get started with our altitude control mission!


In [1]:
#===============================================================================
# Run this cell to start the bridge!
#===============================================================================
import numpy as np
import matplotlib.pyplot as plt
import threading
import asyncio
from ubicoders_vrobots import run_servers
from ubicoders_vrobots.vrobots_clients.vr_client_utils import (
    VirtualRobot,
    vr_client_main,
)
from ubicoders_vrobots.vrobots_msgs.python.states_msg_helper import VRobotState

def run_bridge():
    try:
        asyncio.run(run_servers())
    except KeyboardInterrupt:
        print("Server stopped by user.")

try:
    script_thread = threading.Thread(target=run_bridge)
    script_thread.start()
finally:
    print("Ignition finished")

Ignition finished


Virtual Robots REST Bridge is running @ port 12741
Virtual Robots Bridge is running @ port 12740


## 3. Mission Briefing

Assuming the default mode (without noise), the basic mission is to make the quadrotor maintain its altitude at 20 meters above the ground. To achieve this goal, we will implement a set of 2 PI controllers. One is for the vertical velocity controller, and the other is for position (altitude). Eventually, you can set any desired altitude for the quadrotor to maintain.

The system follows the aerospace engineering axis convention: X front, Y right, Z down. Therefore, 20 meters above the ground in the coordinate system will be (0, 0, -20).

### 3.1 Goals
- Maintain the altitude of 20 meters above the ground (0, 0, -20) in 20 seconds.
- Implement 2 cascade PI controllers for z-axis velocity and position.

### 3.2 Observe the Default Template

The template class `MyVrobotsController` has a `loop` function - similar to Arduino. The `loop()` function will be called whenever the state message arrives from the virtual robot. Then, we can implement anything to control the robot. In this case, we will use one of the built-in functions, `update_cmd_multirotor(sysId, [pwm])`. This function expects a system ID and a PWM list. The system ID comes from the simulation. For now, the PWM values are fixed at 1501 for all 4 motors as this is only for the height control.

Now, try to change the PWM values in the `update_cmd_multirotor()` function and observe how the quadcopter behaves!


In [3]:
class MyVRobotsController:
    def __init__(self, vr: VirtualRobot = None):
        self.vr = vr        

    def loop(self):
        # this is the statese
        states: VRobotState = self.vr.states
        
        # print a few of the states
        print(f"linPos: {states.linPos}") # linear position
        print(f"linVel: {states.linVel}") # linear velocity
        print(f"system ID: {states.sysId}") # systm ID
        print(f"timestamp: {states.timestamp}") # timestamp
    
        # set the pwms (integers) for the 4 motors
        self.vr.update_cmd_multirotor(states.sysId, [1501, 1501, 1501, 1501]) # (system_id, [motor1, motor2, motor3, motor4])

vr_client_main(MyVRobotsController(), duration=10) # interact for 10 seconds

Opened connection
linPos: {
    "x": 0,
    "y": 0,
    "z": 0
}
linVel: {
    "x": 0,
    "y": 0,
    "z": 0
}
system ID: 0
timestamp: 0.0
linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.17302899062633514
}
linVel: {
    "x": 1.1230559948671726e-06,
    "y": -1.9871959011652507e-07,
    "z": 1.803040277081891e-06
}
system ID: 0
timestamp: 1719880216475.0
linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.17302899062633514
}
linVel: {
    "x": 1.5656034975108923e-06,
    "y": -1.7094940574224893e-07,
    "z": 1.9073482917519868e-06
}
system ID: 0
timestamp: 1719880216508.0
linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.17307251691818237
}
linVel: {
    "x": 1.627416622795863e-06,
    "y": -4.153046120336512e-07,
    "z": -0.0014513134956359863
}
system ID: 0
timestamp: 1719880216546.0
linPos: {
    "x": 50.599998474121094,
    "y": 180.63999938964844,
    "z": -0.173116073012352
}
linVel: {
  

: 

## 4. General P-I Controller

As mentioend earlier, there will be 2 PI controllers for velocity and position. In terms of "re-usability" of the code, we will implement a PI control class so we can reuse later.

The PI controller below assumes only 1 axis as the goal is the z-axis position (height or altitutde) control. The key is that we need to saturate the error using `error_max` otherwise, the error integration will fluctuate too much.

In [ ]:
class PIController:
    def __init__(self, kp=1, ki=1, error_max=100):
        self.kp = kp
        self.ki = ki
        self.integral = 0
        self.error_max = error_max

    def error_integrator(self, error):
        self.integral += error
        self.integral = np.clip(self.integral, -self.error_max, self.error_max)
    
    def update(self, error):
        self.error_integrator(error)
        return self.kp * error + self.ki * self.integral

## 5. Putting All Together

Cheat code!

```
self.vel_pi_ctrl = PIController(kp=60, ki=2.5, error_max=100)
self.alt_pi_ctrl = PIController(kp=0.8, ki=0.01, error_max=100)
```

In [ ]:
class MyVRobotsController:

    def __init__(self, vr: VirtualRobot = None):
        self.vr = vr
        self.vel_pi_ctrl = PIController(kp=0, ki=0, error_max=100)
        self.alt_pi_ctrl = PIController(kp=0, ki=0, error_max=100)
 
    def loop(self):
        states: VRobotState = self.vr.states

        # altitude control
        current_altitude = -states.linPos.z # z axis down is positive
        alt_setpoint = 20 # 20 meters above the ground
        error_altitude = alt_setpoint - current_altitude # error is positive if below setpoint
        velocity_setpoint = self.alt_pi_ctrl.update(error_altitude) # velocity setpoint, output of the alt - PI controller
        velocity_setpoint = np.clip(velocity_setpoint, -10, 10)
        print(f"velocity setpoint: {velocity_setpoint}")

        # velocity control
        vel_error = velocity_setpoint - (-states.linVel.z) # error is positive if below setpoint
        throttle_sp =  self.vel_pi_ctrl.update(vel_error)
        throttle = 1200 + throttle_sp
        throttle = np.clip(throttle, 1200, 2000)
        print(f"throttle: {int(throttle)}")
        self.vr.update_cmd_multirotor(0, [throttle, throttle, throttle, throttle])

vr_client_main(MyVRobotsController(), duration=20)